# CTE (Common Table Expression)

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-01-29

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om EMNE i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/t-sql/queries/with-common-table-expression-transact-sql?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/t-sql/queries/with-common-table-expression-transact-sql?view=sql-server-ver16)

## Indholdsfortegnelse 

- [CTE](#CTE)
- [Multiple CTE'er](#multiplecte)
- [Rekursive CTE'er](#recursivecte)
- [Hovedpointer](#Hovedpointer)

## CTE

## <span style="font-size: 14px;">Hvad er en CTE?</span>  
En Common Table Expression (CTE) er et navngivent, temporært tabelresultat af en forespørgsel. 
Du kan danne en CTE med en `WITH`-sætning, og derefter referere den i en `SELECT`-, `INSERT`-, `UPDATE`- eller `DELETE`-sætning. 

CTE'en blev introduceret i ANSI SQL-99 og kom til T-SQL med SQL Server 2005.

Use cases:

- CTE'er anvendes i forespørgsler for at undgå svært-læsbare subqueries og derived tables.
- CTE'er kan gøre scripts nemmere at forstå, fx ved at anvende meningsbærende navngivning og bryde kompleksitet op i forståelige steps.
- CTE'er kan bruges rekursivt, idet det i en CTE er muligt at referere sig selv. Hermed kan CTE'er bruges til at folde fx komplekse parent-child strukturer ud.

In [ ]:
--Som temporær tabel:
CREATE TABLE #TableA (
  Id int NOT NULL,
  Col int NOT NULL
);
INSERT INTO #TableA (Id, Col)
VALUES
(1, 5), 
(2, 6);
SELECT * FROM #TableA;

DROP TABLE #TableA;

--Samme tabel som CTE
WITH TableA AS(
 SELECT * 
 FROM (VALUES 
     (1, 5), 
     (2, 6)) 
     AS v(Id,Col1)
)
SELECT * FROM TableA;

## Multiple CTE'er <a class="anchor" id="multiplecte"></a>

Det er muligt at lave flere CTE'er i samme `WITH`-sætning. Tidligere CTE'er i samme `WITH`-sætning kan refereres. Dette muligggør, at CTE'erne bruges til opdele logikken i et script i sekventielle faser. 

In [34]:
WITH Countries AS(
 SELECT Id, [Name], Parent_Id
 FROM (VALUES 
     (1, 'Europe', NULL), 
     (2, 'Asia', NULL),
     (3, 'EU Europe', 1),
     (4, 'Non-EU Europe', 1),
     (5, 'Austria', 3),
     (6, 'Denmark', 3),
     (7, 'Nepal', 2), 
     (8, 'Norway', 4)) a(Id, [Name], Parent_Id) 
)
, Countries_Parent AS (
  SELECT c.*, cp.[Name] AS ParentName
  FROM Countries c 
  LEFT OUTER JOIN Countries cp ON cp.Id=c.Parent_Id
)

SELECT * FROM Countries_Parent;

(8 rows affected)

Total execution time: 00:00:00.038

Id,Name,Parent_Id,ParentName
1,Europe,NULL,NULL
2,Asia,NULL,NULL
3,EU Europe,1,Europe
4,Non-EU Europe,1,Europe
5,Austria,3,EU Europe
6,Denmark,3,EU Europe
7,Nepal,2,Asia
8,Norway,4,Non-EU Europe


## Rekursive CTE'er <a class="anchor" id="recursivecte"></a>

  
Endelig er det muligt at lave CTE'erne rekursive. 

Mens vi ovenfor så, hvordan vi kunne bruge en CTE til at hente den direkte `parent` i en `parent-child`-relation, så vil vi nu og da ønske at finde den øverste `parent` fremfor den direkte `parent`. Her kommer den rekursive CTE i spil. 

In [35]:
WITH Countries AS(
 SELECT Id, [Name], Parent_Id
 FROM (VALUES 
     (1, 'Europe', NULL), 
     (2, 'Asia', NULL),
     (3, 'EU Europe', 1),
     (4, 'Non-EU Europe', 1),
     (5, 'Austria', 3),
     (6, 'Denmark', 3),
     (7, 'Nepal', 2), 
     (8, 'Norway', 4)) a(Id, [Name], Parent_Id) 
)
, Countries_Parent AS (
    SELECT 
        c.*, 
         0 AS HierarchyLevel,
         c.[Name] AS TopParentName
    FROM Countries AS c
    WHERE Parent_Id IS NULL --Anchor
    UNION ALL
    SELECT 
      c.*,
      HierarchyLevel + 1 AS HierarchyLevel,
      cp.[TopParentName] AS TopParentName
    FROM Countries AS c
    INNER JOIN Countries_Parent cp ON cp.Id=c.Parent_Id
)

SELECT Id, [Name], HierarchyLevel, TopParentName FROM Countries_Parent

(8 rows affected)

Total execution time: 00:00:00.060

Id,Name,HierarchyLevel,TopParentName
1,Europe,0,Europe
2,Asia,0,Asia
7,Nepal,1,Asia
3,EU Europe,1,Europe
4,Non-EU Europe,1,Europe
8,Norway,2,Europe
5,Austria,2,Europe
6,Denmark,2,Europe


### *Tid til opgaver...*

Lav opgave 1, 2 og 3 i [opgavehæftet](Cte.sql).

## Hovedpointer

- CTE'er er temporære og fungerer kun i samme batch, som de er defineret. CTE'er skal ikke droppes efter brug.
- Der kan være mange CTE'er i samme WITH-sætning. CTE'er kan refere tidligere defineret CTE i samme WITH-sætning.
- Det er muligt at lave rekursive CTE'er, som refererer sig selv. Dermed tilbyder CTE'erne kompleks funktionalitet, som ikke findes andre steder i T-SQL.

## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
